In [32]:
import gensim.downloader as api
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.decomposition import PCA

import numpy as np
import pickle
import json
import copy
import pandas as pd
from tqdm import tqdm
import scipy.stats as stats

from Vocab import Vocab
vocab = Vocab('data/vocab.csv', 3)
from get_predictions import get_predictions
from helpers import *

from ChainDataset import ChainDataset

chain_test_set = ChainDataset(
    data_dir='data/',
    segment_file='segments.json',
    chain_file='test_chains.json',
    vectors_file='vectors.json',
    split='test'
)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Initialising vocab from file.


In [3]:
wv_google_300d = api.load('word2vec-google-news-300')
wv_twitter_50d = api.load('glove-twitter-50')


## Cosine similarity

In [4]:
dataset_pred_no_hist, dataset_pred_hist_cp, conditions_inds, condition_seg_hist = get_pred_datasets_orig()

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
processing test

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting predicitons took 56.80236005783081 seconds
loaded model: History
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
par

 97%|█████████▋| 6591/6801 [00:00<00:00, 32678.49it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 39%|███▉      | 2648/6801 [00:00<00:00, 26464.63it/s]

History


100%|██████████| 6801/6801 [00:00<00:00, 27495.80it/s]


In [5]:
stopword_list = stopwords.words('english')

stopword_list.append("-B-")
stopword_list.append("-A-")
stopword_list.append("<unk>")

stemmer = nltk.stem.porter.PorterStemmer()


In [30]:
from sklearn.metrics.pairwise import cosine_similarity

def embed(seg):
    ''' Create embedding vector '''
    embedding = np.zeros((300,))
    length = 0
    for word in seg:

        # only use word if it is not a stopword 
        if word in stopword_list:
            continue
            
        # try to embed word and add it to sentence vector 
        try:
            em = wv_google_300d[word]
            embedding += em
            length += 1
        except KeyError:
            continue
    
    return embedding / length 

def get_something(condition, condition_seg_hist):
    ''' Calculate mean cosine similarity for all conditions '''
    cosine_sim = []
    csh_keys = list(condition_seg_hist[condition])
    count = 0 
    for k in csh_keys:
        
        # ensure current chain is not empty (TODO: fix this maybe? so this doesn't happen)
        if condition_seg_hist[condition][k] == {}:
            continue
            
        # decode first and current sentence 
        first_seg = list(condition_seg_hist[condition][k].values())[0]['first_seg']
        current_seg = list(condition_seg_hist[condition][k].values())[0]['current_seg']
        dec_first_seg = vocab.decode(first_seg)
        dec_current_seg = vocab.decode(current_seg)
    
        # embed words  
        embed_first = embed(dec_first_seg).reshape(1, -1)
        embed_current = embed(dec_current_seg).reshape(1, -1)
        
        # calculate cosine similarity 
        try: 
            cosim = cosine_similarity(embed_first, embed_current)
            cosine_sim.append(cosim)
        except ValueError:
            count += 1
            
    # show how many times no cosine similarity was able to be calculated
    print(count)
    
    # calculate mean and std 
    cosine_sim = np.array(cosine_sim)
    cosine_sim_mean = cosine_sim.mean()
    cosine_sim_std = cosine_sim.std()
    
    return cosine_sim_mean, cosine_sim_std

In [31]:
cosim_hT_nhF = get_something('hT_nhF', condition_seg_hist)
cosim_hT_nhT = get_something('hT_nhT', condition_seg_hist)
cosim_hF_nhF = get_something('hF_nhF', condition_seg_hist)
cosim_hF_nhT = get_something('hF_nhT', condition_seg_hist)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


3


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


7
0
0


In [33]:
stats.f_oneway(cosim_hT_nhF, cosim_hT_nhT, cosim_hF_nhF, cosim_hF_nhT)

F_onewayResult(statistic=0.0013339933011006825, pvalue=0.9999210503125144)